using Pkg;

Pkg.add("LightGraphs");
Pkg.add("Lazy");
Pkg.add("MetaGraphs");
Pkg.add("GraphIO");
Pkg.add("GraphPlot");
Pkg.add("Compose")
Pkg.add("Colors")
Pkg.add("StatsBase")

In [1]:
using LightGraphs
using Lazy: @>, @>>, @as
using MetaGraphs;
using GraphIO;
using GraphPlot;
using Colors;
using DelimitedFiles
using Compose
using StatsBase
using Statistics

In [2]:
edges_raw = readdlm("filtered-imports.txt", ' ', String);

In [3]:
i = 0;
vertex_dict = Dict();
add_vertex_to_dict!(vx) = begin
    global i;
    i += 1;
    vertex_dict[vx] = i;
end;
@>> edges_raw[:] unique foreach(add_vertex_to_dict!)
vertex_dict;

In [4]:
raw_edge_to_prepared_edge(edge) = @> (vertex_dict[edge[2]], vertex_dict[edge[1]]) LightGraphs.SimpleEdge

g = @>> edges_raw eachrow map(raw_edge_to_prepared_edge) SimpleDiGraphFromIterator MetaDiGraph;

In [5]:
add_path_as_prop!((path, vx)) = set_prop!(g, vx, :path, path);
@>> vertex_dict pairs foreach(add_path_as_prop!);

In [6]:
is_bitcoinj_vx(g, vx) = @>> get_prop(g, vx, :path) occursin(".bitcoinj.")
tag_vx_as_bitcoinj!(vx) = @> g set_prop!(vx, :bitcoinj, true)
@>> filter_vertices(g, is_bitcoinj_vx) foreach(tag_vx_as_bitcoinj!);

In [7]:
distances_from_bitcoinj = @as _v g filter_vertices(_v, :bitcoinj) gdistances(g, _v)
unreachable_distance = maximum(distances_from_bitcoinj)
@>> distances_from_bitcoinj filter(x -> x < unreachable_distance) describe

Summary Stats:
Length:         1170
Missing Count:  0
Mean:           1.831624
Minimum:        0.000000
1st Quartile:   1.000000
Median:         2.000000
3rd Quartile:   2.000000
Maximum:        6.000000
Type:           Int64


In [8]:
add_distance_to_bitcoinj_as_prop!((vx, d)) = begin
    reachable = d < unreachable_distance
    if reachable
        set_prop!(g, vx, :bitcoinj_distance, d)
    end
end

@>> distances_from_bitcoinj enumerate foreach(add_distance_to_bitcoinj_as_prop!)

In [171]:
max_bitcoinj_distance = 1
close_to_bitcoinj(g, v) = begin
    if has_prop(g, v, :bitcoinj_distance)
        bitcoinj_distance = get_prop(g, v, :bitcoinj_distance)
        bitcoinj_distance <= max_bitcoinj_distance
    else
        false
    end
end;
close_to_bitcoinj(g, v) = begin
    if has_prop(g, v, :bitcoinj_distance)
        bitcoinj_distance = get_prop(g, v, :bitcoinj_distance)
        bitcoinj_distance <= max_bitcoinj_distance
    else
        false
    end
end;

is_bitcoinj(g, v) = has_prop(g, v, :bitcoinj);

In [172]:
sg, _ = @>> filter_vertices(g, close_to_bitcoinj) collect induced_subgraph(g)
nv(sg) / nv(g)

0.2943396226415094

In [173]:
#maximum(btwns)
scaler(X, a, b) = begin
    max_X = maximum(X)
    min_X = minimum(X)
    return x -> (((b - a) * (x - min_X)) / (max_X - min_X)) + a
end;

scale_to(a, b, X) = @>> X map(scaler(X, a, b));

staircase(n) = x -> @>> ((x * (n-1)) + 1) round(Int);

In [174]:
number_of_colors = max_bitcoinj_distance + 1
range_steps = number_of_colors - 1
color_range = begin
    if range_steps > 1
        range(HSL(colorant"pink"), stop=HSL(colorant"lightblue"), length=range_steps)
    else
        HSL(colorant"pink")
    end;
end;
#color_range = colormap("Reds", number_of_colors, mid=0.5, logscale=true)
#color_range = reverse(color_range)
color_range = @>> color_range vcat([HSL(colorant"tomato")])

In [175]:
sg_bitcoinj_distances = [get_prop(sg, v, :bitcoinj_distance) for v in vertices(sg)];

In [176]:
nodefillc = @>> begin
    sg_bitcoinj_distances
    scale_to(0, 1)
    map(staircase(number_of_colors))
    getindex(color_range)
end

In [177]:
nodesize_steps = 3
nodesize = @>> begin
    [outdegree(sg, v) for v in vertices(sg)]
    scale_to(0, 1)
    map(staircase(nodesize_steps))
end;

In [178]:
# bitcoinj_import_count

bitcoinj_importer(sg, v) = get_prop(sg, v, :bitcoinj_distance) == 1
append_bitcoinj_import_count!(v) = begin
    is_bitcoinj(v) = has_prop(sg, v, :bitcoinj)
    bitcoinj_import_count = @>> v inneighbors(sg) filter(is_bitcoinj) length
    set_prop!(sg, v, :bitcoinj_import_count, bitcoinj_import_count)
end
@>> filter_vertices(sg, bitcoinj_importer) foreach(append_bitcoinj_import_count!)

get_bitcoinj_import_count(v) = begin
    if has_prop(sg, v, :bitcoinj_import_count)
        get_prop(sg, v, :bitcoinj_import_count)
    else
        0
    end
end;
        
bitcoinj_import_count = [get_bitcoinj_import_count(v) for v in vertices(sg)];

In [179]:
using DataFrames
using Query;

In [180]:
append_measure!(f, name, g) = begin 
    measure = @>> g f;
    @>> g vertices foreach(v -> set_prop!(g, v, name, measure[v]));
end;

In [181]:
calc_pagerank(g) = @> g reverse pagerank;
@>> sg append_measure!(calc_pagerank, :pagerank);
pr = [get_prop(sg, v, :pagerank) for v in vertices(sg)];

In [182]:
begin
    df = @>> filter_vertices(sg, :bitcoinj_import_count) map(v->props(sg,v)) DataFrame
    df |>
    @filter(_.bitcoinj_import_count > 4) |>
    @orderby_descending(_.pagerank) |>
    DataFrame
end

,pagerank,path,bitcoinj_import_count,bitcoinj_distance
,Float64,String,Int64,Int64
1,0.01445,bisq.asset.NetworkParametersAdapter,8,1
2,0.0121683,bisq.core.btc.wallet.BtcWalletService,25,1
3,0.00838961,bisq.core.util.coin.BsqFormatter,5,1
4,0.00762545,bisq.core.btc.wallet.BsqWalletService,28,1
5,0.00579844,bisq.desktop.util.GUIUtil,7,1
6,0.00520649,bisq.core.btc.wallet.TradeWalletService,22,1
7,0.0032998,bisq.core.trade.TradeManager,7,1
8,0.00285741,bisq.core.btc.setup.WalletsSetup,24,1
9,0.00279363,bisq.core.btc.wallet.WalletService,37,1


In [184]:
nodesize=1
nodefillc=nodefillc
nodelabelsize = nodesize * 0.85
arrowlengthfrac = 0.01
edgestrokec = colorant"hsla(0, 100%, 0%, 0.15)";
nodelabel=vertices(sg)
C=2
MAXITER=200
layout=(args...)->spring_layout(args...; C=C, MAXITER=MAXITER)
# C default 2.0
# labai didelis C duoda rata, kurio defektai labai idomus
p = gplot(sg,
    layout=layout,
    nodefillc=nodefillc,
    nodesize=nodesize,
    nodelabelsize=nodelabelsize,
    arrowlengthfrac=arrowlengthfrac,
    edgestrokec=edgestrokec,
    nodelabel=nodelabel);

background_color = "white"
give_background(plot) = @>> (context(), rectangle(), fill(background_color)) compose(context(), plot)
@>> begin
    p
    give_background
    draw(SVG("plot.svg", 50cm, 50cm))
end

false

In [91]:
get_path(v) = get_prop(sg, v, :path)
interesting_vs = [414, 424, 426, 465, 455, 441]
interesting_vs = [125,126,131,133,136,138]
@>> interesting_vs map(x-> (x, get_path(x)))

6-element Array{Tuple{Int64,String},1}:
 (125, "bisq.core.btc.setup.WalletConfig")
 (126, "bisq.core.btc.setup.WalletsSetup")
 (131, "bisq.core.btc.wallet.BsqWalletService")
 (133, "bisq.core.btc.wallet.BtcWalletService")
 (136, "bisq.core.btc.wallet.TradeWalletService")
 (138, "bisq.core.btc.wallet.WalletService")